In [1]:
import numpy as np
import pandas as pd
import assets.script as script
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

### Example use case for `script.py` file

In [2]:
x = [np.random.randn(10) for i in range(100)]
y = np.random.randint(1, 100, 100)

In [3]:
df = pd.DataFrame(x, columns=['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
df = df.join(pd.Series(y, name='target'))

In [4]:
X_train, X_test, y_train, y_test = script.split_data(df, target='target')


No columns dropped.

Target values: [ 9 63 67  2 15 18 27 17 93 86  8 68 68 78 17 75 85 29 14 44 83  4 72 47
 92 48 91 33 40 77 42 45 48 93  3 92 14 59 54 91 10  2 40 52 57 96  8 39
 58 75 43 65 75 97 10 17 20 29 29 54  1 65 90 23  8  2 93 80 69 93 59 46
  9 89 36 29 74 43 85 22 39 95 70 67 32  5 32 94  6 93  5 85 42 70  4 73
 71 70 24 37] 

Column(s) remaining: Index(['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
       'nine', 'target'],
      dtype='object')

Data is unscaled.
training sample size:  80
testing sample size:  20


In [5]:
y_pred, y_prob = script.get_predictions(GaussianNB(), X_train, y_train, X_test)

In [6]:
y_pred

array([ 2,  2, 75, 93, 29, 29, 75, 93, 17, 29, 93,  4, 54, 93, 93,  4, 17,
       29, 29, 29])

In [7]:
y_prob

array([[0.00000000e+00, 9.75928594e-01, 0.00000000e+00, ...,
        1.70574170e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.93282313e-01, 0.00000000e+00, ...,
        3.89792581e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.85762009e-01, 0.00000000e+00, ...,
        3.09720463e-02, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.47272462e-10, 0.00000000e+00, ...,
        1.15795119e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.04744800e-08, 0.00000000e+00, ...,
        2.36643719e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.93754687e-03, 0.00000000e+00, ...,
        3.50559344e-07, 0.00000000e+00, 0.00000000e+00]])